In [1]:
import os

In [2]:
# get the data
output_path = "data"

#!wget -O $output_path/LJSpeech-1.1.tar.bz2 https://data.keithito.com/data/speech/LJSpeech-1.1.tar.bz2 
#!tar -xf $output_path/LJSpeech-1.1.tar.bz2 -C $output_path

In [3]:
from src.dataset import load_samples
from src.dataset.dataset import TTSDataset
from src.utils.text import text_to_sequence


148 {'_': 0, '-': 1, '!': 2, "'": 3, '(': 4, ')': 5, ',': 6, '.': 7, ':': 8, ';': 9, '?': 10, ' ': 11, 'A': 12, 'B': 13, 'C': 14, 'D': 15, 'E': 16, 'F': 17, 'G': 18, 'H': 19, 'I': 20, 'J': 21, 'K': 22, 'L': 23, 'M': 24, 'N': 25, 'O': 26, 'P': 27, 'Q': 28, 'R': 29, 'S': 30, 'T': 31, 'U': 32, 'V': 33, 'W': 34, 'X': 35, 'Y': 36, 'Z': 37, 'a': 38, 'b': 39, 'c': 40, 'd': 41, 'e': 42, 'f': 43, 'g': 44, 'h': 45, 'i': 46, 'j': 47, 'k': 48, 'l': 49, 'm': 50, 'n': 51, 'o': 52, 'p': 53, 'q': 54, 'r': 55, 's': 56, 't': 57, 'u': 58, 'v': 59, 'w': 60, 'x': 61, 'y': 62, 'z': 63, '@AA': 64, '@AA0': 65, '@AA1': 66, '@AA2': 67, '@AE': 68, '@AE0': 69, '@AE1': 70, '@AE2': 71, '@AH': 72, '@AH0': 73, '@AH1': 74, '@AH2': 75, '@AO': 76, '@AO0': 77, '@AO1': 78, '@AO2': 79, '@AW': 80, '@AW0': 81, '@AW1': 82, '@AW2': 83, '@AY': 84, '@AY0': 85, '@AY1': 86, '@AY2': 87, '@B': 88, '@CH': 89, '@D': 90, '@DH': 91, '@EH': 92, '@EH0': 93, '@EH1': 94, '@EH2': 95, '@ER': 96, '@ER0': 97, '@ER1': 98, '@ER2': 99, '@EY': 100,

In [4]:

metadata_train, metadata_eval = load_samples(
    'data', 'metadata.csv', True, 50
)

In [5]:
metadata_eval[0]

{'text': 'A molecular change takes place in the nerve of the tentacle,',
 'audio_file': 'data/wavs/LJ025-0134.wav',
 'root_path': 'data'}

In [6]:
text_to_sequence('world!0', cleaner_names=['english_cleaners'])

[60, 52, 55, 49, 41, 2, 63, 42, 55, 52]

In [7]:
eval_ds = TTSDataset(metadata_eval, )

In [8]:
print(eval_ds[1]['mel'].shape)
print(eval_ds[1]['wav'].shape)

(80, 598)
(152989,)


/Users/zhenyishen/Documents/GitHub/TTS_models/src/utils/audio.py:93: FutureWarning: Pass sr=22050, n_fft=1024 as keyword args. From version 0.10 passing these as positional arguments will result in an error
  return librosa.filters.mel(hps.sample_rate, n_fft, n_mels=hps.num_mels, fmin = hps.fmin, fmax = hps.fmax)


In [9]:
eval_ds[1]['mel']

array([[-6.2346873, -6.3425984, -6.6849976, ..., -6.9272847, -6.76982  ,
        -6.9328823],
       [-7.441429 , -6.8091054, -6.923126 , ..., -5.993645 , -6.740462 ,
        -7.5672493],
       [-6.2672606, -6.268371 , -5.8890815, ..., -5.71357  , -6.8414035,
        -5.813919 ],
       ...,
       [-7.523626 , -5.401645 , -4.2046394, ..., -9.337278 , -9.091779 ,
        -9.267428 ],
       [-6.7809362, -5.1289062, -4.078197 , ..., -9.290837 , -9.301757 ,
        -9.637875 ],
       [-6.387214 , -5.385655 , -4.2463613, ..., -9.488182 , -9.325371 ,
        -9.581504 ]], dtype=float32)